In [53]:
# 1. まず*CP.csv からpair、Edisp、 D3のEint、D3なしのEintの組み合わせを得る
import os
import pandas as pd
from IPython.display import display
import glob

# CSVファイルのパス
# CPファイル
csv_file_path = 'ORIENT_RESULT/C1/result_CP.csv'  # 実際のファイルパスに置き換えてください
# dim3ファイル
dim3_file_path = "ORIENT_RESULT/C1/dim3_pair_C1.csv" # 実際のファイルパスに置き換えてください

data_dir = "/".join(csv_file_path.split("/")[0:-1])
pair_dir = data_dir + "/PAIR"
os.makedirs(pair_dir, exist_ok=True)

# CSVファイルを読み込む
df = pd.read_csv(csv_file_path)
df['Pair'] = df['File Name'].str.extract(r'pair_(\d+)')

# 'Etotal (D3)'を計算するための関数を定義
def calculate_etotal(row, df, col):
    if row['count'] == 1:
        count_2 = df[(df['Pair'] == row['Pair']) & (df['count'] == 2)][col].values[0]
        count_3 = df[(df['Pair'] == row['Pair']) & (df['count'] == 3)][col].values[0]
        return (row[col] - count_2 - count_3)*2625.5
    return None



# 'Etotal (D3)'を計算して新しいカラムに追加
df['Etotal (D3)'] = df.apply(lambda row: calculate_etotal(row, df, 'SCF Done Value (D3-corrected)'), axis=1)
df['Etotal (B3LYP)'] = df.apply(lambda row: calculate_etotal(row, df, 'SCF Done Value (B3LYP)'), axis=1)

# 必要なカラムのみを選択して新しいDataFrameを作成
result_df = df[['Pair', 'Etotal (D3)', 'Etotal (B3LYP)']].dropna()
result_df["Edisp"] = result_df["Etotal (D3)"] - result_df["Etotal (B3LYP)"] 

# 結果を新しいCSVファイルに保存
output_csv_path = data_dir + '/CP_result_processed.csv'  # 出力先のファイルパスを設定してください
result_df.to_csv(output_csv_path, index=False)


In [54]:
# dim3の結果も必要なものを抽出する
df_dim3 = pd.read_csv(dim3_file_path)
df_dim3['Pair'] = df_dim3['File Name'].str.extract(r'pair_(\d+)')
result_df_dim3 = df_dim3[['Pair', 'Electrostatic Energy', 'Induction Energy']]
dim3_output_csv_path = data_dir + '/CP_result_processed_dim3.csv' 
result_df_dim3.to_csv(dim3_output_csv_path, index=False)

In [55]:
# CSVファイルを読み込む
cp_processed_df = pd.read_csv(output_csv_path)
cp_processed_dim3_df = pd.read_csv(dim3_output_csv_path)

# 'Pair'列を基準にして横方向に結合（concatenate）
concat_df = pd.concat([cp_processed_df.set_index('Pair'), cp_processed_dim3_df.set_index('Pair')], axis=1, join='inner').reset_index()
concat_df["Replusion"] = concat_df["Etotal (B3LYP)"] - concat_df["Electrostatic Energy"] - concat_df["Induction Energy"]
concat_df = concat_df[['Pair', 'Electrostatic Energy', 'Induction Energy', 'Replusion', 'Edisp', 'Etotal (D3)', 'Etotal (B3LYP)']]

# 結果を新しいCSVファイルに保存
output_csv_path = data_dir + '/processed_DMP_results.csv'  # 出力先のファイルパスを設定してください
concat_df.to_csv(output_csv_path, index=False)

# ソートしたものを保存
sorted_concat_df = concat_df.sort_values(by='Electrostatic Energy')
sorted_concat_df_path = data_dir + '/processed_DMP_results_sorted.csv'
sorted_concat_df.to_csv(sorted_concat_df_path, index=False)

display(sorted_concat_df)

,Pair,Electrostatic Energy,Induction Energy,Replusion,Edisp,Etotal (D3),Etotal (B3LYP)
4,14,-9.629612,-4.729041,11.471916,-21.532251,-24.418988,-2.886737
10,4,-9.600678,-4.724231,11.474141,-21.527525,-24.378293,-2.850768
5,15,-9.403342,-4.851696,12.345512,-21.894044,-23.803571,-1.909526
8,2,-9.400899,-4.845419,12.322614,-21.886693,-23.810397,-1.923704
11,5,-5.228211,-3.855994,30.268577,-84.045931,-62.861559,21.184372
7,1,-5.215209,-3.854833,30.281982,-84.058796,-62.846856,21.211940
12,6,-4.057418,-1.030073,6.146618,-11.030251,-9.971124,1.059127
0,10,-4.055040,-1.029757,6.159939,-11.034976,-9.959834,1.075142
6,16,0.031608,-0.187973,0.571457,-4.043533,-3.628441,0.415092
9,3,0.034370,-0.188124,0.572259,-4.044583,-3.626078,0.418505


## Groupラベルの追加 (一度目視の確認が必要、上で出力された結果に基づきn_clustersの値を適宜変える)

In [56]:
import pandas as pd
from sklearn.cluster import KMeans

# CSVファイルのパス
csv_file_path = sorted_concat_df_path  

# CSVファイルを読み込む
df = pd.read_csv(csv_file_path)

# KMeansクラスタリングを使用して7つのグループに分ける
kmeans = KMeans(n_clusters=7)
df['Group'] = kmeans.fit_predict(df[['Electrostatic Energy', 'Induction Energy', 'Replusion', 'Edisp']])
# df['Group'] = kmeans.fit_predict(df[['Electrostatic Energy']])

# Groupカラムを'Electrostatic Energy'の大きい順に1から7まで名付ける
group_means = df.groupby('Group')['Electrostatic Energy'].mean().sort_values(ascending=True)
new_group_mapping = {old: new for new, old in enumerate(group_means.index, start=1)}
df['Group'] = df['Group'].map(new_group_mapping)

df = df[['Pair', 'Group', 'Electrostatic Energy', 'Induction Energy', 'Replusion', 'Edisp', 'Etotal (D3)', 'Etotal (B3LYP)']]


# 結果を新しいCSVファイルに保存
grouped_csv_path = data_dir + '/processed_DMP_results_sorted_grouped.csv'  
df.to_csv(grouped_csv_path, index=False)

print("等価であることを確認してください")
display(df)

等価であることを確認してください


,Pair,Group,Electrostatic Energy,Induction Energy,Replusion,Edisp,Etotal (D3),Etotal (B3LYP)
0,14,1,-9.629612,-4.729041,11.471916,-21.532251,-24.418988,-2.886737
1,4,1,-9.600678,-4.724231,11.474141,-21.527525,-24.378293,-2.850768
2,15,1,-9.403342,-4.851696,12.345512,-21.894044,-23.803571,-1.909526
3,2,1,-9.400899,-4.845419,12.322614,-21.886693,-23.810397,-1.923704
4,5,2,-5.228211,-3.855994,30.268577,-84.045931,-62.861559,21.184372
5,1,2,-5.215209,-3.854833,30.281982,-84.058796,-62.846856,21.211940
6,6,3,-4.057418,-1.030073,6.146618,-11.030251,-9.971124,1.059127
7,10,3,-4.055040,-1.029757,6.159939,-11.034976,-9.959834,1.075142
8,16,4,0.031608,-0.187973,0.571457,-4.043533,-3.628441,0.415092
9,3,4,0.034370,-0.188124,0.572259,-4.044583,-3.626078,0.418505


## 中心+group_nのgjfファイル作成 (Groupが等価なことを確認したら)

## PAIRフォルダに"pair_n.gjf"を追加
## PAIRフォルダに"central.gjf"と名付けたファイルを追加、このファイルは中心分子だけの座標を持つファイルでCONSTANT以降の行は消す(改行もすべて)

In [142]:
# 等価なグループのMを持つ行を追加
import shutil
df = pd.read_csv(grouped_csv_path)
groups = df["Group"].unique()
pairs = df["Pair"].unique()
result_path =  pair_dir + "/result"
os.makedirs(result_path, exist_ok=True)
central_file = pair_dir + "/central.gjf"

file_path = pair_dir + "/*.gjf"
paths = [p for p in glob.glob(file_path)]

with open(central_file, 'r') as file:
    lines = file.readlines()

    # 最後の行が空白行でなければ、空白行を追加する
    if lines[-1] != '\n':
        lines.append('\n')

# 変更をファイルに書き戻す
with open(central_file, 'w') as file:
    file.writelines(lines)

for path in paths:
    if 'pair' in path:
        pair_n = int(path.split("pair_")[-1].split(".gjf")[0])
        group_n = df[df["Pair"]==pair_n]["Group"].iloc[-1]
        # コピー先のファイル名を設定
        destination_file = os.path.join(result_path, f'central_+group_{group_n}.gjf')
        # 同じ名前のファイルがすでに存在する場合は処理をスキップ
        if not os.path.exists(destination_file):
            # ファイルをコピー
            shutil.copy(central_file, destination_file)
        else:
            print(f"File '{destination_file}' already exists. Skipping the copy process.")
        
#         pathのfileのM行をdestination_fileの末尾に付け足していく
        with open(path, 'r') as src, open(destination_file, 'a') as dst:
            for line in src:
                if line.endswith(" M\n"):
                    dst.write(line)


File 'ORIENT_RESULT/C1/PAIR/result/central_+group_2.gjf' already exists. Skipping the copy process.
File 'ORIENT_RESULT/C1/PAIR/result/central_+group_1.gjf' already exists. Skipping the copy process.
File 'ORIENT_RESULT/C1/PAIR/result/central_+group_3.gjf' already exists. Skipping the copy process.
File 'ORIENT_RESULT/C1/PAIR/result/central_+group_5.gjf' already exists. Skipping the copy process.
File 'ORIENT_RESULT/C1/PAIR/result/central_+group_7.gjf' already exists. Skipping the copy process.
File 'ORIENT_RESULT/C1/PAIR/result/central_+group_6.gjf' already exists. Skipping the copy process.
File 'ORIENT_RESULT/C1/PAIR/result/central_+group_4.gjf' already exists. Skipping the copy process.
File 'ORIENT_RESULT/C1/PAIR/result/central_+group_1.gjf' already exists. Skipping the copy process.
File 'ORIENT_RESULT/C1/PAIR/result/central_+group_1.gjf' already exists. Skipping the copy process.


In [140]:
dst

<_io.TextIOWrapper name='ORIENT_RESULT/C1/PAIR/result/central_+group_1.gjf' mode='a' encoding='UTF-8'>

In [125]:
df[df["Pair"]==pair_n]["Group"].iloc[-1]

1

In [127]:
group

1

In [ ]:
 source_file = '/PAIR/central.gjf'
destination_dir = '/PAIR/result'

# コピー先のファイル名を設定
destination_file = os.path.join(destination_dir, f'central_group_{group_n}.gjf')

# 同じ名前のファイルがすでに存在する場合は処理をスキップ
if not os.path.exists(destination_file):
    # ファイルをコピー
    shutil.copy(source_file, destination_file)
else:
    print(f"File '{destination_file}' already exists. Skipping the copy process.")
